# Naive Bayes - Trabalho

## Questão 1

Implemente um classifacor Naive Bayes para o problema de predizer a qualidade de um carro. Para este fim, utilizaremos um conjunto de dados referente a qualidade de carros, disponível no [UCI](https://archive.ics.uci.edu/ml/datasets/car+evaluation). Este dataset de carros possui as seguintes features e classe:

** Attributos **
1. buying: vhigh, high, med, low
2. maint: vhigh, high, med, low
3. doors: 2, 3, 4, 5, more
4. persons: 2, 4, more
5. lug_boot: small, med, big
6. safety: low, med, high

** Classes **
1. unacc, acc, good, vgood





In [13]:
import csv
 
def loadCsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [x for x in dataset[i]]
    return dataset

In [16]:
filename = 'carData.csv'
dataset = loadCsv(filename)
print(('O arquivo {0} foi carregado com {1} linhas').format(filename, len(dataset)))

O arquivo carData.csv foi carregado com 1728 linhas


In [21]:
import pandas as pd
df = pd.DataFrame(dataset)
print(df)

          0      1      2     3      4     5      6
0     vhigh  vhigh      2     2  small   low  unacc
1     vhigh  vhigh      2     2  small   med  unacc
2     vhigh  vhigh      2     2  small  high  unacc
3     vhigh  vhigh      2     2    med   low  unacc
4     vhigh  vhigh      2     2    med   med  unacc
5     vhigh  vhigh      2     2    med  high  unacc
6     vhigh  vhigh      2     2    big   low  unacc
7     vhigh  vhigh      2     2    big   med  unacc
8     vhigh  vhigh      2     2    big  high  unacc
9     vhigh  vhigh      2     4  small   low  unacc
10    vhigh  vhigh      2     4  small   med  unacc
11    vhigh  vhigh      2     4  small  high  unacc
12    vhigh  vhigh      2     4    med   low  unacc
13    vhigh  vhigh      2     4    med   med  unacc
14    vhigh  vhigh      2     4    med  high  unacc
15    vhigh  vhigh      2     4    big   low  unacc
16    vhigh  vhigh      2     4    big   med  unacc
17    vhigh  vhigh      2     4    big  high  unacc
18    vhigh 

In [22]:
labels = {'buiyng': df[0],
          'maint': df[1],
          'doors': df[2],
          'persons': df[3],
          'lug_boot': df[4],
          'safety': df[5],
          'classes': df[6]}

In [53]:
df = pd.DataFrame(labels)

In [54]:
#Transform data into numerical
df = df.replace('vhigh', 4)
df = df.replace('high', 3)
df = df.replace('med', 2)
df = df.replace('low', 1)
df = df.replace('more', 6)
df = df.replace('big', 3)
df = df.replace('small', 1)
df = df.replace('unacc', 1)
df = df.replace('acc', 2)
df = df.replace('good', 3)
df = df.replace('vgood', 4)
df = df.replace('5more', 6)
df = df.replace('2', 2)
df = df.replace('3', 3)
df = df.replace('4', 4)

In [71]:
import random
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

In [74]:
splitRatio = 0.7
newDF = df.values.tolist()
train, teste = splitDataset(newDF, splitRatio)

1209

In [57]:

#Separating using dataframe
'''
dados = df.drop(["classes"],axis=1)
classe = df["classes"]
train_x, train_y
'''




## Questão 2
Crie uma versão de sua implementação usando as funções disponíveis na biblioteca SciKitLearn para o Naive Bayes ([veja aqui](http://scikit-learn.org/stable/modules/naive_bayes.html)) 

## Questão 3

Analise a acurácia dos dois algoritmos e discuta a sua solução.